# Divide and Conquer Algorithm
Given a problem of size $n$ devide the problem into $a$ subproblems each of size $n/b$, solve each one recursively by further devide it intil a base case (*conquer*) and combine the subproblems to form the solution for the original problem.

![Working-of-Divide-and-Conquer-Algorithm.webp](Working-of-Divide-and-Conquer-Algorithm.webp)

## Master Method
The Master Method determines the time complexity of the total problem based on the characteristics of the particular devide and conquer paradimg.
Each divide and conquer algorithm has the following characteristics:
- $a$ is the number of problems that the original problem is devided into.
- $b$ is the size of each of the sub-problems (Note that $a$ could be different than $b$)
- a base case when the recursion is stoped when the size of the sub-problems reaches a minimum case
- $C(n)$ is the time complexity for combining the $a$ subproblems 
- $D(n)$ is the tme complexity for dividing the problem of size $n$ into $a$ subproblems each of size $n/m$
Then we can form the following recursion equation:
$$
T(n) = 
\begin{cases}
\mathcal{\Theta}(1) & \quad , \,\, \text{if $n$ smaller than a cutoff} \\
a*T\left(\frac{n}{b}\right)+D(n)+C(n) & \quad , \,\, \text{otherwise}
\end{cases}
$$ 
The general solution to this recursion is called the *Master Method* and is given as:
- let $\epsilon = \log_{b}a$
- let $D(n)+C(n) = \Theta(n^c)$ for some $c$
- then we can prove that:
$$
T(n) = 
\begin{cases}
\Theta(n^{\epsilon}) \,\, &, \,\, \text{if}\,\, \epsilon>c \\
\Theta(n^{\epsilon}\log_2n) \,\, &, \,\, \text{if}\,\, \epsilon=c \\
\Theta(n^c) \,\, &, \,\, \text{if}\,\, \epsilon<c 

\end{cases}
$$ 
#### Examples of the Master Method

For the merge-sort algorithm:
- we divide an array of  size $n$ into two arrays $(a=2)$
- each of half size of the original $(b=2)$.  
- the divide step takes $\mathcal{O}(1)$ and the combine step takes $\mathcal{O}(n)$, so $c=1$. 
- $\epsilon = \log _b a = 1$ and so $c = \epsilon =1$
- this is the second case in master method and so the merge-sort algorithm runs in $\mathcal{O}(n\log_2n)$ time

### Karatsuba's Multiplication Algorithm
Given two integer $n_1,n_2$ we want to design an algorithm to calculate their product $n_1\cdot n_2$ in an efficient way.
- let $n$ be the number of binary digits of $n_1$, and $m$ the number of binary digits of $n_2$, and let $n>m$. 
- the traditional multiplication algorithm runs in $\mathcal{O}(m(n+m))$

In [1]:
def binary(n:int)-> list:
    '''return a list of bits representing the binary form of an integer n in the forn [b_{n-1}, b_{n-2},..., b_0]
    '''
    
    return [int(b) for b in bin(n)[2:]]

def padd_left(a: list, n: int)-> list:
    '''pad a bit-list representation a with zeros such that the list has n bits, but is the same number
    '''
    
    return [0]*max((n-len(a)),0) + a

def padd_right(a: list, n: int)-> list:
    '''pad a bit-list representation a with zeros such that the list has n bits, but is the number a*2**(a-len(n))
    '''
    
    return a+[0]*max((n-len(a)),0)

def make_equal_size(a:list, b:list)-> list:
    '''takes two bit-lists representations of integers and pads the smaller sized one with zeros
    '''
    n = max(len(a),len(b))
    return padd_left(a,n), padd_left(b,n)

def add(a_bin:list, b_bin:list)->list:

    a_bin, b_bin = make_equal_size(a_bin, b_bin)

    carry = 0 
    c = [0]*len(a_bin)
    for i in range(len(a_bin)-1,-1,-1):
        c[i] = (a_bin[i]+b_bin[i]+carry)%2
        carry = (a_bin[i]+b_bin[i]+carry)//2
    # if the last carry is one then c has one more digit than a,b
    if carry==1:
        c = [1]+c

    return c

def subtr(a_bin:list, b_bin:list)->list:

    a_bin, b_bin = make_equal_size(a_bin, b_bin)

    carry = 0 
    c = [0]*len(a_bin)
    for i in range(len(a_bin)-1,-1,-1):
        c[i] = (a_bin[i]- b_bin[i]-carry)%2
        carry = int((a_bin[i]- b_bin[i]- carry)<0)
    
    return c

def mult(a, b):
    a_bin, b_bin = binary(a), binary(b)
    a_bin, b_bin = make_equal_size(a_bin, b_bin)

    mult_bin = [0]*(2*len(a_bin))
    temp = a_bin
    for j in range(len(a_bin)-1,-1,-1):
        if b_bin[j]:
            mult_bin = add(mult_bin, temp)
        temp = padd_right(temp, len(temp)+1)
    
    return mult_bin

In [2]:
def make_int(b_bin:list):
    return int('0b'+''.join(map(str,b_bin)), base=2)

def find_power(n:int):
    
    k=0
    while n>2**k:
        k+=1
    return k

find_power(5)

3

In [10]:
def prepare_karatsuba(n1: int, n2: int):
    n1_bin, n2_bin = binary(n1), binary(n2)
    n1_bin, n2_bin = make_equal_size(n1_bin, n2_bin)
    n_bits = 2**find_power(len(n1_bin))
    n1_bin, n2_bin = padd_left(n1_bin, n_bits), padd_left(n2_bin, n_bits)
    
    return n1_bin, n2_bin

def devide_four_mul(a_bin, b_bin):
    if len(a_bin)==1:
        return [a_bin[0]*b_bin[0]]
    n = len(a_bin)
    a1 = a_bin[:n//2]
    a2 = a_bin[n//2:]
    b1 = b_bin[:n//2]
    b2 = b_bin[n//2:]
    # print(f'a1={a1}, a2={a2}, b1={b1}, b2={b2}')
    p1 = devide_four_mul(a1, b1)
    p4 = devide_four_mul(a2, b2)
    p2 = devide_four_mul(a1, b2)
    p3 = devide_four_mul(a2, b1)
    # print(f'p1={p1}, p2={p2}, p3={p3}, p4={p4}')
    
    p1 = padd_right(p1, len(p1)+n)
    p2 = padd_right(add(p2, p3), len(p2)+n//2)
    # print(f'after padding')
    # print(f'p1={p1}, p2+p3={p2} p4={p4}')

    return add(add(p1, p2), p4)


In [9]:
n1 , n2 = 323,234
n1_b, n2_b = prepare_karatsuba(n1, n2)
# print(n1_b, n2_b)
m = devide_four_mul(n1_b, n2_b)
# print(m)
make_int(m) , n1*n2

(75454, 75582)

In [148]:
def karatsuba(a_bin, b_bin):
    if len(a_bin)==1:
        return [a_bin[0]*b_bin[0]]
    n = len(a_bin)
    a1 = a_bin[:n//2]
    a2 = a_bin[n//2:]
    b1 = b_bin[:n//2]
    b2 = b_bin[n//2:]
    # print(f'a1={a1}, a2={a2}, b1={b1}, b2={b2}')

    p1 = karatsuba(a1, b1)
    p4 = karatsuba(a2, b2)
    a12, b12 = make_equal_size(add(a1,a2),add(b1,b2))
    p = karatsuba(a12, b12)
    p23 = subtr(subtr(p,p1), p4)
    # print(f'p1={p1}, p={p}, p23={p23} p4={p4}')

    p1 = padd_right(p1, len(p1)+n)
    p23= padd_right(p23, len(p23)+n//2)
    # print(f'after padding')
    # print(f'p1={p1}, p2+p3={p23} p4={p4}')

    return add(add(p1, p23), p4)